# local

In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sql("create table if not exists src (key int, value string) using hive")
spark.sql("load data local inpath '/opt/spark/examples/src/main/resources/kv1.txt' into table src")

23/11/22 06:43:19 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/22 06:43:19 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/22 06:43:20 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/11/22 06:43:20 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/11/22 06:43:20 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/22 06:43:20 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/22 06:43:20 WARN HiveMetaStore: Location: hdfs://master/user/hive/warehouse/src specified for non-external table:src


DataFrame[]

23/11/22 06:43:21 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [5]:
spark.sql("select * from src").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows



In [6]:
spark.sql("select count(*) from src").show()

+--------+
|count(1)|
+--------+
|     500|
+--------+



In [7]:
sqldf = spark.sql("select key, value from src where key < 10 order by key")
stringsds = sqldf.rdd.map(lambda row: f"key: {row.key}, value: {row.value}")
for record in stringsds.collect():
    print(record)

key: 0, value: val_0
key: 0, value: val_0
key: 0, value: val_0
key: 2, value: val_2
key: 4, value: val_4
key: 5, value: val_5
key: 5, value: val_5
key: 5, value: val_5
key: 8, value: val_8
key: 9, value: val_9


In [8]:
from pyspark.sql import Row

Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

In [9]:
spark.sql("select * from records r join src s on r.key = s.key").show()

+---+------+---+------+
|key| value|key| value|
+---+------+---+------+
|  2| val_2|  2| val_2|
|  4| val_4|  4| val_4|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  8| val_8|  8| val_8|
|  9| val_9|  9| val_9|
| 10|val_10| 10|val_10|
| 11|val_11| 11|val_11|
| 12|val_12| 12|val_12|
| 12|val_12| 12|val_12|
| 15|val_15| 15|val_15|
| 15|val_15| 15|val_15|
| 17|val_17| 17|val_17|
| 18|val_18| 18|val_18|
| 18|val_18| 18|val_18|
| 19|val_19| 19|val_19|
| 20|val_20| 20|val_20|
| 24|val_24| 24|val_24|
| 24|val_24| 24|val_24|
+---+------+---+------+
only showing top 20 rows



In [10]:
spark.stop()

# remote